In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../../')

In [3]:
import datetime
import arrow
import os
import pandas as pd
import warnings
import time

from tqdm import tqdm

from util.file_util import StockTwitsFileReader
from util.stocktwits_helper import (
    get_est_market_datetime, get_daily_sentiment_count_df,
    get_all_stocktwits_tickers)

In [4]:
from arrow.factory import ArrowParseWarning

In [5]:
# Disable Arrow warnings per https://github.com/crsmithdev/arrow/issues/612
warnings.simplefilter("ignore", ArrowParseWarning)

In [6]:
stock_twits_reader = StockTwitsFileReader()

In [ ]:
ticker = 'NVDA'

In [ ]:
start_date = '2014-01-01'
end_date = '2018-03-30'

In [ ]:
df = stock_twits_reader.read_twit_file_in_range(ticker, start_date, end_date, cols='default')

In [ ]:
daily_sentiment_df = get_daily_sentiment_count_df(df)

In [ ]:
df[['date_est', 'created_at_est', 'mkt_date']].head(10)

## Save down sentiment time series

In [7]:
start_date = '2016-01-01'
end_date = '2019-08-10'

#### Run this for all tickers

In [8]:
from config import STOCKTWITS_TICKER_LIST

In [9]:
ticker_list = STOCKTWITS_TICKER_LIST
len(ticker_list)

502

In [10]:
# ticker_list = get_all_stocktwits_tickers()
# ticker_list = [
#     'UBER',  # Uber
#     'LYFT',  # Lyft
#     'WORK',  # Slack
#     'ZM',  # Zoom
#     'PINS',  # Pinterest
#     'CHWY',  # Chewy
#     'CRWD',  # Crowdstrike
# ]

In [11]:
root_dir = stock_twits_reader.get_root_dir()
save_dir = os.path.join(root_dir, 'processed', 'sentiment_summary_prelim_new')
save_dir

'/Users/seung-jae_bang/Personal/Research/Stock_Sentiment/data/Stocktwits/processed/sentiment_summary_prelim_new'

In [12]:
save_file_format = '{ticker}_sentiment_summary.pkl'
save_path_format = os.path.join(save_dir, save_file_format)
save_path_format

'/Users/seung-jae_bang/Personal/Research/Stock_Sentiment/data/Stocktwits/processed/sentiment_summary_prelim_new/{ticker}_sentiment_summary.pkl'

In [13]:
for t in tqdm(ticker_list):
    save_file_path = save_path_format.format(ticker=t)
    
    if os.path.exists(save_file_path):
        continue
    
    tic = time.time()
    
    raw_df = stock_twits_reader.read_twit_file_in_range(t, start_date, end_date, cols='default')
    daily_sentiment_df = get_daily_sentiment_count_df(raw_df)
    daily_sentiment_df.to_pickle(save_file_path)
    
    toc = time.time()
#     print('Run for {} took {} mins'.format(t, (toc - tic) / 60))

100%|██████████| 502/502 [3:02:23<00:00, 21.80s/it]
